## Introduction to `robotoc` 
# 2: Robot Models

## The goal of this chapter
Here, we learn how to create robot models (`robotoc::Robot`) from URDF packages.  
Note that, `robotoc::Robot` is just a wrapper of [`Pinocchio`](https://github.com/stack-of-tasks/pinocchio)'s API.  
So, if you want to dive in deeper, please check [`Pinocchio`'s documentation](https://stack-of-tasks.github.io/pinocchio/).

## What is URDF?
URDF (universal robot description format) describes the robot.  
We need very little knowledge on URDF as long as we use off-the-shelf URDF files with `robotoc`.  
So don't worry even if you do not know well about URDF!  
You can see off-the-shelf URDF packages in this repository's `urdf` folder.    
As we can see there, a URDF package is composed of urdf file (`~.urdf`) and mesh files (e.g., `~.stl`, `~.dae`, or `~.obj`)   
The urdf file describes the tree structure of the multi-link robot and mesh files contain visual and collision information.   
If you'd like to know more, please see, e.g., the ROS tutorials http://wiki.ros.org/urdf/Tutorials.  

## Simplest example: a robot manipulator `iiwa14`

First, we construct `robotoc.Robot` (`robotoc::Robot` in C++) with a simple example.  
Note that C++'s `robotoc::Robot` has more rich methods than the following Python counterpart.

In [1]:
import robotoc
path_to_urdf = "urdf/iiwa_description/urdf/iiwa14.urdf"
iiwa14 = robotoc.Robot(path_to_urdf)

<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for boost::shared_ptr<hpp::fcl::CollisionGeometry> already registered; second conversion method ignored.


We can see the information of the robot as 

In [2]:
print(iiwa14)

robot model:
  name: iiwa14
  base joint: fixed base
  contacts:  dimq = 7,   dimv = 7,   dimu = 7,   dim_passive = 0

  frames:
    frame 0
      name: universe
      parent joint id: 0

    frame 1
      name: root_joint
      parent joint id: 0

    frame 2
      name: world
      parent joint id: 0

    frame 3
      name: world_iiwa_joint
      parent joint id: 0

    frame 4
      name: iiwa_link_0
      parent joint id: 0

    frame 5
      name: iiwa_joint_1
      parent joint id: 1

    frame 6
      name: iiwa_link_1
      parent joint id: 1

    frame 7
      name: iiwa_joint_2
      parent joint id: 2

    frame 8
      name: iiwa_link_2
      parent joint id: 2

    frame 9
      name: iiwa_joint_3
      parent joint id: 3

    frame 10
      name: iiwa_link_3
      parent joint id: 3

    frame 11
      name: iiwa_joint_4
      parent joint id: 4

    frame 12
      name: iiwa_link_4
      parent joint id: 4

    frame 13
      name: iiwa_joint_5
      parent joint id: 5


The printed information is as follows:
- `dimq`: the dimension of the configuration
- `dimv`: the dimension of the generalized velocity (not always the same as `dimq`)
- `dimu`: the dimension of the input torque
- `dim_passive`: the dimension of the passive joints including the floating base
- `frames`: frames of the robot. you can use to design the cost function and constraints on the end-effector.
- `joints`: each joint info.
- `effort limit`: limits of joint torques
- `velocity limit`: limits of joint velocities
- `lower position limit`, `upper position limit`: limits of joint positions

You can individually get some of them as

In [3]:
iiwa14.dimq()

7

In [4]:
iiwa14.dimv()

7

In [5]:
iiwa14.dimu()

7

In [6]:
iiwa14.dim_passive()

0

In [7]:
iiwa14.joint_effort_limit()

array([300., 300., 300., 300., 300., 300., 300.])

In [8]:
iiwa14.joint_velocity_limit()

array([10., 10., 10., 10., 10., 10., 10.])

In [9]:
iiwa14.lower_joint_position_limit()

array([-2.96705973, -2.0943951 , -2.96705973, -2.0943951 , -2.96705973,
       -2.0943951 , -3.05432619])

In [10]:
iiwa14.upper_joint_position_limit()

array([2.96705973, 2.0943951 , 2.96705973, 2.0943951 , 2.96705973,
       2.0943951 , 3.05432619])

Next, let's visualize them with some random configuration

In [11]:
steps = 10
q = [iiwa14.generate_feasible_configuration() for i in range(steps)]
dt = 1.0

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf, viewer_type='meshcat')
viewer.display(dt, q)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


You can also call simple forward kinematics with the robot object.

In [12]:
end_effector_frame_name = 'iiwa_link_ee_kuka'
end_effector_frame = iiwa14.frame_id(end_effector_frame_name)
for e in q:
    iiwa14.forward_kinematics(e)
    print('q: ', e)
    print('frame position: ', iiwa14.frame_position(end_effector_frame))
    print('frame rotation: ', iiwa14.frame_rotation(end_effector_frame))
    print('frame placement (SE3): ', iiwa14.frame_placement(end_effector_frame))

q:  [-1.31871141  0.22606882 -0.13412859  0.53981327 -0.80238509  0.0561336
  2.76251419]
frame position:  [0.01416113 0.06745666 1.2705805 ]
frame rotation:  [[ 0.86937004 -0.49215756 -0.04445982]
 [ 0.48481408  0.83204927  0.26953537]
 [-0.09566111 -0.25588072  0.96196362]]
frame placement (SE3):    R =
   0.86937  -0.492158 -0.0444598
  0.484814   0.832049   0.269535
-0.0956611  -0.255881   0.961964
  p = 0.0141611 0.0674567   1.27058

q:  [ 2.46975125  0.56846796  1.28946594 -1.50125171  0.63476609 -2.02611543
 -1.57061534]
frame position:  [-0.51983827  0.02303756  0.72559083]
frame rotation:  [[-0.15418868 -0.97586634 -0.154631  ]
 [ 0.35749202 -0.20100018  0.91202982]
 [-0.92110005  0.08534532  0.37985638]]
frame placement (SE3):    R =
-0.154189 -0.975866 -0.154631
 0.357492    -0.201   0.91203
  -0.9211 0.0853453  0.379856
  p = -0.519838 0.0230376  0.725591

q:  [-2.15271116  1.27413261 -2.0373073  -0.41492315 -2.19686771 -1.63861786
  3.04775645]
frame position:  [-0.5981124

## Quadrupedal robot `ANYmal`

Second, we try a quadruped robot example.  
The quadrupedal robot typically involves contacts. 
So we specify it via frame names or frame indices.  
Here, we do it via names.

In [23]:
contact_frames = ['LF_FOOT', 'LH_FOOT', 'RF_FOOT', 'RH_FOOT']

Further, there several types of contacts.  
Quadruped robots have point contacts, so we specify it as 

In [24]:
contact_types = [robotoc.ContactType.PointContact for _ in contact_frames]

We specify the time-step parameter (or weight parameters) of the Baumgarte's stabilization method.  
This is to take the accurate rigid contacts into account.  
Please see detail, e.g., [this paper](https://www.researchgate.net/publication/234610391_A_Parametric_Study_on_the_Baumgarte_Stabilization_Method_for_Forward_Dynamics_of_Constrained_Multibody_Systems).
Appropriate parameter depends on the time step of MPC formulation.  
So some experiment is necessary.   
In my experience, 2~10 times of time step of MPC work well.
Here, we set it as 0.04.

In [25]:
baumgarte_time_step = 0.04

Finally, we construct the robot.  
Since the quadruped robot has a floating base, we specify it in the constructor.

In [26]:
path_to_urdf = 'urdf/anymal_b_simple_description/urdf/anymal.urdf'
anymal = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                       contact_frames, contact_types, baumgarte_time_step)

We can see the detail of the robot model as 

In [27]:
print(anymal)

robot model:
  name: anymal
  base joint: floating base
  contacts:point contact:
  contact frame id: 12
  parent joint id: 4
  Baumgarte's weights on (velocity, position): (50, 625)
point contact:
  contact frame id: 22
  parent joint id: 7
  Baumgarte's weights on (velocity, position): (50, 625)
point contact:
  contact frame id: 32
  parent joint id: 10
  Baumgarte's weights on (velocity, position): (50, 625)
point contact:
  contact frame id: 42
  parent joint id: 13
  Baumgarte's weights on (velocity, position): (50, 625)
  dimq = 19,   dimv = 18,   dimu = 12,   dim_passive = 6

  frames:
    frame 0
      name: universe
      parent joint id: 0

    frame 1
      name: root_joint
      parent joint id: 1

    frame 2
      name: base
      parent joint id: 1

    frame 3
      name: LF_HAA
      parent joint id: 2

    frame 4
      name: LF_HIP
      parent joint id: 2

    frame 5
      name: LF_HFE
      parent joint id: 3

    frame 6
      name: LF_THIGH
      parent joint i

We can also display the robot.  

In [28]:
steps = 10
q = [anymal.generate_feasible_configuration() for i in range(steps)]
dt = 1.0

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf,
                                        base_joint_type=robotoc.BaseJointType.FloatingBase, 
                                        viewer_type='meshcat')
viewer.display(dt, q)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


## Humanoid robot `iCub`

Third, we try a humanoid robot example.  
Humanoid robots have typically two surface contacts. 

In [29]:
contact_frames = ['l_sole', 'r_sole']
contact_types = [robotoc.ContactType.SurfaceContact for i in contact_frames]

baumgarte_time_step = 0.05

path_to_urdf = 'urdf/icub_description/urdf/icub.urdf'
icub = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                     contact_frames, contact_types, baumgarte_time_step)

We can see the detail of the robot model as 

In [30]:
print(icub)

robot model:
  name: iCub
  base joint: floating base
  contacts:surface contact:
  contact frame id: 26
  parent joint id: 7
  Baumgarte's weights on (velocity, position): (40, 400)
surface contact:
  contact frame id: 46
  parent joint id: 13
  Baumgarte's weights on (velocity, position): (40, 400)
  dimq = 36,   dimv = 35,   dimu = 29,   dim_passive = 6

  frames:
    frame 0
      name: universe
      parent joint id: 0

    frame 1
      name: root_joint
      parent joint id: 1

    frame 2
      name: base_link
      parent joint id: 1

    frame 3
      name: base_fixed_joint
      parent joint id: 1

    frame 4
      name: root_link
      parent joint id: 1

    frame 5
      name: l_hip_pitch
      parent joint id: 2

    frame 6
      name: l_hip_1
      parent joint id: 2

    frame 7
      name: l_hip_roll
      parent joint id: 3

    frame 8
      name: l_hip_2
      parent joint id: 3

    frame 9
      name: l_leg_ft_sensor
      parent joint id: 3

    frame 10
     

and display the robot

In [32]:
steps = 10
q = [icub.generate_feasible_configuration() for i in range(steps)]
dt = 1.0

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf,
                                        base_joint_type=robotoc.BaseJointType.FloatingBase, 
                                        viewer_type='meshcat')
viewer.display(dt, q)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
